In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline
import spacy

from pdfminer.high_level import extract_text
# import pdfplumber
import re
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer, util

In [2]:
# For BERT based NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [3]:
# For spaCy based NER

# !python -m spacy download en_core_web_trf
import spacy_transformers
nlp_spacy_trf = spacy.load("en_core_web_trf")
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#### pdfminer.six

In [76]:
a = nlp_spacy_trf('Hi my name is XYZ')
for _ in a.sents:
    print(_)

Hi my name is XYZ


In [21]:
# text = extract_text('../data/ashok_leyland_concall_transcript.pdf')
# text = extract_text('../data/AGM_Transcript_Revised.pdf')

#### pdfplumber: Better results in pdf extraction -> Check later

with pdfplumber.open("../data/AGM_Transcript_Revised.pdf") as file:
    text = file.pages[10].extract_text()

In [22]:
# Idea -> Creating tokens as Points of Interest (POI) in the text extracted from the PDF
# and using these tokens to extract Entities of Interest (EOI)
string = 'I now invite Mr. A.V. Mani Sundaram, CLID IN30163741521740.<mask>A.V. Mani Sundaram:'
rege = r'(<mask>(.*)<mask>(.*?):)|(<mask>(.*?):)'

re.findall(rege, string)[0][-1]

'A.V. Mani Sundaram'

In [9]:
sentences = [
    " Thank you very much. We will now begin the question and answer session. We will wait for a moment while the question can be assembled. ",
    " Dear Shareholders, good morning and a very warm welcome to the Annual General Meeting of City   Union   Bank   Limited   for   FY'2021   through   video   conferencing   or   other   audio-visual facility. As a reminder, for the smooth conduct of the meeting, the members will be in the mute mode and audio and video will be opened when they will speak at the AGM as per the pre-registration. Please note that as per the requirements, the proceedings of the annual general meeting will be recorded  and  available on the website  of the Bank.  I now hand over the proceedings to Shri R. Mohan, the Chairman of City Union Bank Limited. Thank you and over ",
    "ask, question and answer",
    ]
sentence_embeddings = model.encode(sentences)

for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

# doc1 = nlp_spacy_trf(u)
# doc2 = nlp_spacy_trf(u'ask, question and answer')
print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]))
print(util.pytorch_cos_sim(sentence_embeddings[1], sentence_embeddings[2]))
print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[2]))

Sentence:  Thank you very much. We will now begin the question and answer session. We will wait for a moment while the question can be assembled. 
Embedding: [ 0.06344783  0.11630968  0.576895   -0.05625724 -0.38268468 -0.48887476
  0.01866261  0.22074395  1.7286305  -1.1689817   0.23807026  0.46779093
  0.6454454   0.04822326  0.6396784  -0.4240498  -0.37041032  0.05249115
 -0.37773827 -0.68010116  0.27323407 -0.3261881  -0.31360036  1.0595748
  0.5199989  -0.02574236  0.06939525  0.38551262 -0.05191303  0.8558748
  0.02803756 -0.2245875  -0.35413888  0.756421    0.1604854   0.04266265
  0.7750803  -0.16950706 -0.53763425 -0.20274737 -0.11885174 -0.61751074
  0.58929455 -0.3541158  -0.34435958 -0.22837614 -0.14803869 -0.04544735
 -1.3836113  -0.3095633  -0.8094896   0.27334154  0.16742463 -0.2938665
 -0.53182346 -0.38980478  0.33786342  0.7349423   0.71322507 -0.11272969
 -0.16684145  0.25240716 -0.1544134  -0.90846854  1.1012332  -0.2501768
  0.27942017 -0.02505153 -0.05847352 -0.914

#### NOTE: The processed data's accuracy will depend on the quality of pdf extraction 

In [23]:
# Step 1: Creating POI
text = text.replace('\n\n', '<mask>').replace('\n', '<m>').replace('\x0c', '')

In [24]:
# Step 2: Extracting EOI

all_eoi = re.findall(r'<mask>(.*?):', text)
all_eoi = [str(_.strip()) + ':' for _ in all_eoi]

list_of_entities = []
for entity in all_eoi:
    match_group = re.findall(r'(<mask>(.*)<mask>(.*?):)|(<mask>(.*?):)', entity)
    non_empty_group = [ele for inner_group in match_group for ele in inner_group if ele]
    # print(non_empty_group)
    if match_group:
        if len(non_empty_group[0]) < 2:
            pass
        else:
#             print(match_group[0][1])
            list_of_entities.append(non_empty_group[-1])
#     list_of_entities.extend(re.findall(r'<mask><mask>(.*)<mask><mask>(.*?):', entity)[-1])

# Approach: Preliminary weeding out entities based on length of the entity
# Assumption: A typical name would not exceed 40 characters
filtered_entity_list = [filtered_element for filtered_element in list_of_entities if len(filtered_element) <= 40]
print(filtered_entity_list)


['MANAGEMENT', 'MODERATOR', 'Moderator', 'Jinesh Gandhi', 'Gopal Mahadevan', 'But coming to the Quarterly Results now', 'seeing happening', 'Moderator', 'Kapil Singh', 'Gopal Mahadevan', 'Kapil Singh', 'Gopal Mahadevan', 'Kapil Singh', 'Gopal Mahadevan', 'Moderator', 'Gunjan Pritiyani', 'Gopal Mahadevan', 'Gunjan Pritiyani', 'Gopal Mahadevan', 'Gunjan Pritiyani', 'Gopal Mahadevan', 'Moderator', 'Hitesh Goel', 'Gopal Mahadevan', 'Hitesh Goel', 'Gopal Mahadevan', 'K. M. Balaji', 'Hitesh Goel', 'Gopal Mahadevan', 'Moderator', 'Sachin Trivedi', 'Gopal Mahadevan', 'Sachin Trivedi', 'Gopal Mahadevan', 'Moderator', 'Shyam Sundar Sriram', 'Gopal Mahadevan', 'Shyam Sundar Sriram', 'Gopal Mahadevan', 'K. M. Balaji', 'Gopal Mahadevan', 'K. M. Balaji', 'Gopal Mahadevan', 'Shyam Sundar Sriram', 'Gopal Mahadevan', 'Moderator']


In [8]:
# Tentative Approach: Trying to filter out PER entities  (discarded)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

for each_element in filtered_entity_list:
    ner = nlp(each_element.replace('.', ''))
#     print(ner)
# Cons: Does not work as expected; Moderator, Management does not fit under any entity; breaks down single words
# and provides entities (is this expected?) -> It is expected

NameError: name 'tokenizer' is not defined

In [25]:
# Tentative Approach: Trying to filter out PER entities  (using spaCy based NEW approach)
# Selected approach

ner_list = []
for word_entity in nlp_spacy_trf(" | ".join(filtered_entity_list)).ents:
    name_property = word_entity.text, word_entity.label_
    ner_list.append(name_property)
# print(ner_list)

/home/aum/Desktop/projects/con_trans_summary/con_summary_env/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [26]:
# Step 3: Extracting location of all EOI

entity_span = []
for element in filtered_entity_list:
    for find_result in re.finditer(str(element) + ':', text):
        span = (element, int(find_result.start()), int(find_result.end()))
        entity_span.append(span)
entity_span = list(set(entity_span))
entity_span = sorted(entity_span, key=lambda span_list: span_list[1])
# print(entity_span)

In [27]:
# Step 4: Filtering out irrelevant EOI

filtered_entity_span = []
for entity_of_interest in entity_span:
    name = entity_of_interest[0].lower()
    for ner_result in ner_list:
        if ("management" in name) or (
            "moderator" in name) or (
            "invites" in name) or (
            "special" in name) or (
            "board" in name) or (
            "director" in name):
            filtered_entity_span.append(entity_of_interest)
            break
        elif name in ner_result[0].lower():
            filtered_entity_span.append(entity_of_interest)
            break
        else:
            pass
print(filtered_entity_span)

[('MANAGEMENT', 85, 96), ('MODERATOR', 237, 247), ('Moderator', 337, 347), ('Jinesh Gandhi', 511, 525), ('Gopal Mahadevan', 629, 645), ('Moderator', 9805, 9815), ('Kapil Singh', 10047, 10059), ('Gopal Mahadevan', 10491, 10507), ('Kapil Singh', 14650, 14662), ('Gopal Mahadevan', 14912, 14928), ('Kapil Singh', 15668, 15680), ('Gopal Mahadevan', 15764, 15780), ('Moderator', 16514, 16524), ('Gunjan Pritiyani', 16645, 16662), ('Gopal Mahadevan', 17089, 17105), ('Gunjan Pritiyani', 21856, 21873), ('Gopal Mahadevan', 22363, 22379), ('Gunjan Pritiyani', 24746, 24763), ('Gopal Mahadevan', 24934, 24950), ('Moderator', 26092, 26102), ('Hitesh Goel', 26203, 26215), ('Gopal Mahadevan', 26456, 26472), ('Hitesh Goel', 27557, 27569), ('Gopal Mahadevan', 27676, 27692), ('K. M. Balaji', 27855, 27868), ('Hitesh Goel', 27932, 27944), ('Gopal Mahadevan', 28229, 28245), ('Moderator', 33334, 33344), ('Sachin Trivedi', 33446, 33461), ('Gopal Mahadevan', 33851, 33867), ('Sachin Trivedi', 35743, 35758), ('Gopal

In [58]:
# Step 5: Extracting relevant information based on the EOI

desired_columns = ["Sr.No.", "Name", "GroupOfSentences"]
information_frame = pd.DataFrame(None, columns=desired_columns)
for sequence, entity_information in enumerate(filtered_entity_span):
    start_index = entity_information[1]
    end_index = entity_information[2]
    if entity_information[0].lower() == "management":
        management_string = text[
            end_index: filtered_entity_span[sequence+1][1]
        ].replace("<mask>", "|").replace("<m>", "|")
        management_list = management_string.split("|")
        
        frame = pd.DataFrame([[sequence, entity_information[0], management_string]], columns=desired_columns)

    elif ("directors" in entity_information[0].lower()) and ("board" in entity_information[0].lower()):
        director_string = text[
            end_index: filtered_entity_span[sequence+1][1]
        ].replace("<mask>", "|").replace("<m>", "|")
        director_list = director_string.split("|")

        frame = pd.DataFrame([[sequence, entity_information[0], director_string]], columns=desired_columns)

    else:
        try:
            relevant_string = text[
                end_index: filtered_entity_span[sequence+1][1]
            ].replace("<mask>", " ").replace("<m>", " ")
        except IndexError:
            relevant_string = text[
                end_index:
            ].replace("<mask>", " ").replace("<m>", " ")
    
        frame = pd.DataFrame([[sequence, entity_information[0], relevant_string]], columns=desired_columns)
    
    information_frame = pd.concat([information_frame, frame], axis=0)
    information_frame.reset_index(drop=True, inplace=True)

In [59]:
# information_frame.to_excel("../data/ashok_leyland_concall_transcript.xlsx", index=False)

In [60]:
information_frame

,Sr.No.,Name,GroupOfSentences
0,0,MANAGEMENT,MR. GOPAL MAHADEVAN – WHOLE TIME DIRECTOR |& ...
1,1,MODERATOR,MR. JINESH GANDHI– MOTILAL OSWAL FINANCIAL S...
2,2,Moderator,"Ladies and gentlemen, Good day, and..."
3,3,Jinesh Gandhi,"Thank you, Mallika. Good morning, everyone. O..."
4,4,Gopal Mahadevan,"Thank you Jinesh and a very warm, good mornin..."
5,5,Moderator,Thank you very much. We will now begin the qu...
6,6,Kapil Singh,"Yes. So, firstly, I wanted to check we have s..."
7,7,Gopal Mahadevan,"Okay, you have almost covered, on ..."
8,8,Kapil Singh,"Sure sir. So, sir overall, how much is the co..."
9,9,Gopal Mahadevan,"No, I will, see the other expenses there ..."


In [61]:
management_string

' MR. GOPAL MAHADEVAN – WHOLE TIME DIRECTOR |& CHIEF FINANCIAL OFFICER.|MR. K. M. BALAJI – VICE PRESIDENT (CORPORATE |FINANCE).|'

In [62]:
# Step 6: Identify conversations initiated by the Management

management_corpus = management_string.lower()

information_frame["type_of_conversation"] = None

question_count = 0
outlook_count = 0
for i, conversation_initiator in enumerate(information_frame["Name"].values):
    if conversation_initiator.strip().lower() in management_corpus:
        if question_count != 0:
            if outlook_count == 0:
                information_frame.loc[i, "type_of_conversation"] = "Outlook"
                outlook_count += 1
            else:
                information_frame.loc[i, "type_of_conversation"] = "Answer"
        else:
            information_frame.loc[i, "type_of_conversation"] = "Outlook"
            outlook_count += 1
            
    elif (conversation_initiator.strip().lower() != "management") and (
          conversation_initiator.strip().lower() != "moderator") and (
          "special invites" not in conversation_initiator.strip().lower()) and (
          "board" not in conversation_initiator.strip().lower()) and (
          "directors" not in conversation_initiator.strip().lower()):
        information_frame.loc[i, "type_of_conversation"] = "Question"
        question_count += 1

information_frame.loc[:information_frame[information_frame['type_of_conversation'] == "Outlook"].index.values.tolist()[0]-1, "type_of_conversation"] = None        

information_frame

,Sr.No.,Name,GroupOfSentences,type_of_conversation
0,0,MANAGEMENT,MR. GOPAL MAHADEVAN – WHOLE TIME DIRECTOR |& ...,None
1,1,MODERATOR,MR. JINESH GANDHI– MOTILAL OSWAL FINANCIAL S...,None
2,2,Moderator,"Ladies and gentlemen, Good day, and...",None
3,3,Jinesh Gandhi,"Thank you, Mallika. Good morning, everyone. O...",None
4,4,Gopal Mahadevan,"Thank you Jinesh and a very warm, good mornin...",Outlook
5,5,Moderator,Thank you very much. We will now begin the qu...,None
6,6,Kapil Singh,"Yes. So, firstly, I wanted to check we have s...",Question
7,7,Gopal Mahadevan,"Okay, you have almost covered, on ...",Answer
8,8,Kapil Singh,"Sure sir. So, sir overall, how much is the co...",Question
9,9,Gopal Mahadevan,"No, I will, see the other expenses there ...",Answer


In [18]:
# Step 7: Filtering Answers and constructing a corpus

answer_corpus = [''.join(
    information_frame[
        information_frame["type_of_conversation"] == "Answer"
    ]["GroupOfSentences"].values.tolist()
)]

# Splitting the statements on a full stop (.)
answer_list = answer_corpus[0].split(".")
print(len(answer_list))

364


In [19]:
# Step 8: Conducting sentiment analysis for each sentence in the answer corpus

classifier = pipeline('sentiment-analysis', model='ProsusAI/finbert')

In [21]:
sentiment_matrix = pd.DataFrame(None, columns=['Label', 'Score'])
sentiment_analysis_results = classifier(answer_list)

for idx, per_sentence_sentiment in enumerate(sentiment_analysis_results):
    sentiment_matrix.loc[idx, ['Label', 'Score']] = (per_sentence_sentiment['label'], per_sentence_sentiment['score'])


In [24]:
# Step 9: Segregating statements by their sentiment >= 0.8

positive_corpus = []
neutral_corpus = []
negative_corpus = []

filtered_sentiment_matrix = sentiment_matrix[sentiment_matrix["Score"] >= 0.8]

for _index in filtered_sentiment_matrix.index.tolist():
    if sentiment_matrix.loc[_index, "Label"] == "positive":
        positive_corpus.append(answer_list[_index])
    elif sentiment_matrix.loc[_index, "Label"] == "negative":
        negative_corpus.append(answer_list[_index])
    else:
        neutral_corpus.append(answer_list[_index])

relevant_sentence_pool = [positive_corpus, neutral_corpus, negative_corpus]
strong_sentences = pd.DataFrame(
    (each_corpus for each_corpus in itertools.zip_longest(*relevant_sentence_pool)),
    columns=['Positive', 'Neutral', 'Negative'])

strong_sentences

,Positive,Neutral,Negative
0,"On the positive side, again, we're seeing rea...","As a remainder, all participant lines ...","But then suddenly, we had the lockdown again,..."
1,"But again, on the positive side, we believe t...",And there will be an opportunity for you to a...,Pricing continues to be a challen...
2, The second bit is on the Light Commercial V...,Should you need assistance during the confere...,I've been mentioning that over the last few q...
3,"But we, at the moment, believe that with the ...",Please note that this conference is being rec...,"At the same time, what we have been doing is ..."
4,"So, LCV business has been doing well it has b...",I now hand the conference over to Mr,"So, practically if you look at it schools wer..."
...,...,...,...
187,None,"Yes, because at the level of volume, it is b...",None
188,None,"See, understand one thing, inflation Ashok Le...",None
189,None,"So, there are a multiple ways to do this",None
190,None,"So, if I may take that as the last question",None


# -------------------------------------------------------------------------------------------

In [15]:
# Idea: Try to retain context.
# 1. Parse through the content and bin Questions and their supposed answers (Questions and all answers 
# following the questions.)
# 2. For each question -> Perform sentiment analysis on the answer corpus.
# 3. Select the top sentiment sentences > 0.8
# 4. Cluster sentences based on the sentiment (positive - positive - neutral - positive)
# 5. Cluster sentences based on the sentiment (negative - negative - neutral - negative)

In [68]:
outlook_indexes = information_frame[information_frame['type_of_conversation'] == "Outlook"].index.tolist()
question_indexes = information_frame[information_frame['type_of_conversation'] == "Question"].index.tolist()
answer_indexes = information_frame[information_frame['type_of_conversation'] == "Answer"].index.tolist()
indexes_to_exclude = information_frame[information_frame['type_of_conversation'].isnull()].index.tolist()

In [74]:
# Create groups of Question and answers
qna_groups = []
for i, question_index in enumerate(question_indexes):
    grouping = [question_index]
    for j, answer_index in enumerate(answer_indexes):
        try:
            if (answer_index > question_index) and (answer_index < question_indexes[i+1]):
                grouping.append(answer_index)
        except IndexError:
            if answer_index > question_index:
                grouping.append(answer_index)
    qna_groups.append(grouping)
print(qna_groups)

[[6, 7], [8, 9], [10, 11], [13, 14], [15, 16], [17, 18], [20, 21], [22, 23, 24], [25, 26], [28, 29], [30, 31], [33, 34], [35, 36, 37, 38, 39, 40], [41, 42]]


In [ ]:
summary_matrix = pd.DataFrame(None, columns=['Question', 'Answer'])
summaries = []
for i, each_group in enumerate(groups):
    summary_matrix.loc[i, 'Question'] = question_indexes[i]
    summary_matrix.loc[i, 'Answer'] = answer_indexes[]
    